# Extracting all the links

## Let's wrap up the previous code

In [1]:
import jl_io as io

genres = io.from_file('../data/genres')
styles = io.from_file('../data/styles')
decades = io.from_file('../data/decades')

In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from functools import reduce

class SongExtractor:
    BASE_URL = "https://www.ultimate-guitar.com/explore?&type[]=Chords";
    MAX_NUM_PAGES = 20
    
    def __init__(self):
        pass
    
    def get_all_songs(self, genre, style, decade):
        links = self.get_all_filter_song_links(genre['pattern'], style['pattern'], decade['pattern'])
        result = [ self.link_to_song_dict(link,genre,style,decade) for link in links]
        
        return result
    
    def link_to_song_dict(self,link, genre, style, decade):
        return {
            "name": link.contents[0],
            "url": link['href'],
            "genre": genre["name"],
            "style": style["name"],
            "decade": decade["name"]
        }
    
    def get_all_filter_song_links(self,genreFilter,styleFilter,decadeFilter):
        list_of_list = [self.get_links_single_page(genreFilter, styleFilter, decadeFilter, f'&page={page}')
                  for page in range(1,self.MAX_NUM_PAGES+1)]
        
        return reduce(lambda list1, list2: [*list1, *list2], list_of_list)
    
    def get_links_single_page(self,genreFilter,styleFilter,decadeFilter, pageFilter):
        driver = self.create_chrome_driver();
        driver.get(f'{self.BASE_URL}{genreFilter}{styleFilter}{decadeFilter}')

        self.click_on_accept_cookies(driver)

        soup = BeautifulSoup(driver.page_source, 'lxml')

        return soup.findAll('a', {"class":"_2KJtL _1mes3 kWOod"})
    
    def click_on_accept_cookies(self, driver):
        button = driver.find_element_by_xpath('//button[contains(text(), "thanks")]')

        button.click()  
    
    def create_chrome_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--headless')
        
        driver = webdriver.Chrome("./chromedriver", options=options)
        return driver
        

In [18]:
%%time
extractor = SongExtractor()

songs = extractor.get_all_songs(genres[0], styles[0], decades[0])

CPU times: user 1.5 s, sys: 154 ms, total: 1.65 s
Wall time: 1min 11s


In [7]:
len(songs)

1000

In [11]:
songs[1:5]

[{'name': 'Radioactive',
  'url': 'https://tabs.ultimate-guitar.com/tab/imagine-dragons/radioactive-chords-1171909',
  'genre': 'Rock',
  'style': 'Pop Rock',
  'decade': '2010s'},
 {'name': 'Say Something (ver\xa03)',
  'url': 'https://tabs.ultimate-guitar.com/tab/a_great_big_world/say_something_chords_1443639',
  'genre': 'Rock',
  'style': 'Pop Rock',
  'decade': '2010s'},
 {'name': 'Counting Stars',
  'url': 'https://tabs.ultimate-guitar.com/tab/onerepublic/counting-stars-chords-1233464',
  'genre': 'Rock',
  'style': 'Pop Rock',
  'decade': '2010s'},
 {'name': 'Million Reasons',
  'url': 'https://tabs.ultimate-guitar.com/tab/lady-gaga/million-reasons-chords-1884102',
  'genre': 'Rock',
  'style': 'Pop Rock',
  'decade': '2010s'}]

## Calculate number of combinations 

In [17]:
%%time
combinations = []

for decade in decades:
    for genre in genres:
        for style in styles:
            combinations.append( (decade, genre, style))
     

CPU times: user 3.97 ms, sys: 35 µs, total: 4.01 ms
Wall time: 3.99 ms


In [13]:
len(combinations)

14700

In [15]:
combinations[0]

({'code': '2010', 'name': '2010s', 'pattern': '&&decade[]=2010'},
 {'code': '4', 'name': 'Rock', 'pattern': '&&genres[]=4'},
 {'code': '24', 'name': 'Pop Rock', 'pattern': '&&subgenres[]=24'})

## Try to improve performace

In [8]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from functools import reduce

class ImprovedSongExtractor:
    BASE_URL = "https://www.ultimate-guitar.com/explore?&type[]=Chords";
    MAX_NUM_PAGES = 20
    
    def __init__(self):
        self.driver = self.create_chrome_driver()
        self.first_time = True
    
    def get_all_songs(self, genre, style, decade):
        links = self.get_all_filter_song_links(genre['pattern'], style['pattern'], decade['pattern'])
        result = [ self.link_to_song_dict(link,genre,style,decade) for link in links]
        
        return result
    
    def link_to_song_dict(self,link, genre, style, decade):
        return {
            "name": link.contents[0],
            "url": link['href'],
            "genre": genre["name"],
            "style": style["name"],
            "decade": decade["name"]
        }
    
    def get_all_filter_song_links(self,genreFilter,styleFilter,decadeFilter):
        list_of_list = [self.get_links_single_page(genreFilter, styleFilter, decadeFilter, f'&page={page}')
                  for page in range(1,self.MAX_NUM_PAGES+1)]
        
        return reduce(lambda list1, list2: [*list1, *list2], list_of_list)
    
    def get_links_single_page(self,genreFilter,styleFilter,decadeFilter, pageFilter):
        self.driver.get(f'{self.BASE_URL}{genreFilter}{styleFilter}{decadeFilter}')
        
        if self.driver.page_source == '<html><head></head><body></body></html>':
            raise Exception('Denegation error')

        if self.first_time:
            self.click_on_accept_cookies()
            self.first_time = False

        soup = BeautifulSoup(self.driver.page_source, 'lxml')

        return soup.findAll('a', {"class":"_2KJtL _1mes3 kWOod"})
    
    def click_on_accept_cookies(self):
        try:
            button = self.driver.find_element_by_xpath('//button[contains(text(), "thanks")]')

            button.click()
        except:
            print('cookies banner not found. Ignored')
            
    
    def create_chrome_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--headless')
        
        driver = webdriver.Chrome("./chromedriver", options=options)
        return driver

In [22]:
%%time
extractor = ImprovedSongExtractor()

songs = extractor.get_all_songs(genres[0], styles[0], decades[0])

CPU times: user 1.44 s, sys: 43.3 ms, total: 1.49 s
Wall time: 17.1 s


## Saving last combination index 

If we get an error during the operation (unexpected error, service denegation etc.), we should be able to restart from the last point. Let's try to save the index inside combinations some way 

In [27]:
number = 88

In [29]:
io.to_file(number,'../data/lastIndex.pickle')

In [30]:
io.from_file('../data/lastIndex.pickle')

88

It works using `pickle`!

In [33]:
! rm '../data/lastIndex.pickle'

In [34]:
! ls ../data

decades genres  styles


## AllCombinationsSongExtractor 

In [24]:
import jl_io as io


class AllCombinationsSongExtractor:
    START_INDEX_FILE_PATH = '../data/nextIndex.pickle'
    def __init__(self, singleSongExtractor):
        self.extractor = singleSongExtractor
        
        self.genres = io.from_file('../data/genres')
        self.styles = io.from_file('../data/styles')
        self.decades = io.from_file('../data/decades')
        
        self.combinations = []

        for decade in self.decades:
            for genre in self.genres:
                for style in self.styles:
                    self.combinations.append( {"decade": decade, "genre": genre, "style": style })
        
    def extract(self, reset=False):      
        startIndex = 1
        
        if not reset:
            try:
                startIndex = io.from_file(self.START_INDEX_FILE_PATH)
            except:
                pass
        
        return self.extract_from(startIndex)
            
    def extract_from(self, startIndexBase1):
        songs = []
        
        for index, combination in enumerate(self.combinations[startIndexBase1 - 1:]):
            new_extracted_songs = []
            genre = combination["genre"]
            style = combination["style"]
            decade = combination["decade"]
            
            try:
                new_extracted_songs = self.extractor.get_all_songs(genre,style,decade)
            except Exception as e: 
                print(f'Error in ({genre["name"]},{style["name"]},{decade["name"]})')
                print(e)
                return songs
            
            base_1_index = index + startIndexBase1
            songs.extend(new_extracted_songs)
            
            number_of_songs = len(songs)
            print(f'Extracted {index+startIndexBase1} of {len(self.combinations)} ({genre["name"]},{style["name"]},{decade["name"]}). {number_of_songs} in total')
  
            io.to_file(index + startIndexBase1 , self.START_INDEX_FILE_PATH)
        
        return songs  
        

In [12]:
all_combination_extractor = AllCombinationsSongExtractor(ImprovedSongExtractor)

In [77]:
all_songs = all_combination_extractor.extract()

Extracted 1 of 14700 (Rock,Pop Rock,2010s)
Extracted 2 of 14700 (Rock,Singer-songwriter,2010s)
Extracted 3 of 14700 (Rock,Alternative Rock,2010s)
Extracted 4 of 14700 (Rock,Indie Rock,2010s)
Extracted 5 of 14700 (Rock,Indie Pop,2010s)
Extracted 6 of 14700 (Rock,Pop Punk,2010s)
Extracted 7 of 14700 (Rock,Hard Rock,2010s)
Extracted 8 of 14700 (Rock,Contemporary Folk,2010s)
Extracted 9 of 14700 (Rock,Folk Rock,2010s)
Extracted 10 of 14700 (Rock,Folk Pop,2010s)
Extracted 11 of 14700 (Rock,Contemporary Country,2010s)
Extracted 12 of 14700 (Rock,Country Pop,2010s)
Extracted 13 of 14700 (Rock,Electropop,2010s)
Extracted 14 of 14700 (Rock,Indie Folk,2010s)
Extracted 15 of 14700 (Rock,Americana,2010s)
Extracted 16 of 14700 (Rock,Punk Rock,2010s)
Extracted 17 of 14700 (Rock,Power Pop,2010s)
Extracted 18 of 14700 (Rock,Country Rock,2010s)
Extracted 19 of 14700 (Rock,Dance-pop,2010s)
Extracted 20 of 14700 (Rock,Blues Rock,2010s)
Extracted 21 of 14700 (Rock,Alternative Country,2010s)
Extracted 22 o

Extracted 173 of 14700 (Folk,Sunshine Pop,2010s)
Extracted 174 of 14700 (Folk,Merseybeat,2010s)
Extracted 175 of 14700 (Folk,Sophisti-pop,2010s)
Extracted 176 of 14700 (Folk,Traditional Pop,2010s)
Extracted 177 of 14700 (Folk,Ska Punk,2010s)
Extracted 178 of 14700 (Folk,Neo Soul,2010s)
Extracted 179 of 14700 (Folk,Alternative R&b,2010s)
Extracted 180 of 14700 (Folk,Rockabilly,2010s)
Extracted 181 of 14700 (Folk,Bluegrass,2010s)
Extracted 182 of 14700 (Folk,Melodic Hardcore,2010s)
Extracted 183 of 14700 (Folk,Garage Rock,2010s)
Extracted 184 of 14700 (Folk,Folk Punk,2010s)
Extracted 185 of 14700 (Folk,Progressive Country,2010s)
Extracted 186 of 14700 (Folk,Psychedelic Folk,2010s)
Extracted 187 of 14700 (Folk,Bro-country,2010s)
Extracted 188 of 14700 (Folk,Yacht Rock,2010s)
Extracted 189 of 14700 (Folk,Pop Reggae,2010s)
Extracted 190 of 14700 (Folk,Alternative Dance,2010s)
Extracted 191 of 14700 (Folk,Progressive Bluegrass,2010s)
Extracted 192 of 14700 (Folk,Ska,2010s)
Extracted 193 of 1

Extracted 343 of 14700 (Country,Pop Rap,2010s)
Extracted 344 of 14700 (Country,Psychedelic Pop,2010s)
Extracted 345 of 14700 (Country,Blue-eyed Soul,2010s)
Extracted 346 of 14700 (Country,Southern Rock,2010s)
Extracted 347 of 14700 (Country,Contemporary Christian Music,2010s)
Extracted 348 of 14700 (Country,Alternative Metal,2010s)
Extracted 349 of 14700 (Country,Post-hardcore,2010s)
Extracted 350 of 14700 (Country,Roots Rock,2010s)
Extracted 351 of 14700 (Country,Heavy Metal,2010s)
Extracted 352 of 14700 (Country,Neo-psychedelia,2010s)
Extracted 353 of 14700 (Country,Outlaw Country,2010s)
Extracted 354 of 14700 (Country,Glam Rock,2010s)
Extracted 355 of 14700 (Country,Latin Pop,2010s)
Extracted 356 of 14700 (Country,Skate Punk,2010s)
Extracted 357 of 14700 (Country,Post-punk,2010s)
Extracted 358 of 14700 (Country,Vocal Jazz,2010s)
Extracted 359 of 14700 (Country,Lo-fi Indie,2010s)
Extracted 360 of 14700 (Country,Progressive Pop,2010s)
Extracted 361 of 14700 (Country,Indietronica,2010s

Extracted 496 of 14700 (Electronic,Rap Rock,2010s)
Extracted 497 of 14700 (Electronic,Comedy Rock,2010s)
Extracted 498 of 14700 (Electronic,Funk,2010s)
Extracted 499 of 14700 (Electronic,Nu Metal,2010s)
Extracted 500 of 14700 (Electronic,Celtic Punk,2010s)
Extracted 501 of 14700 (Rhythm And Blues,Pop Rock,2010s)
Extracted 502 of 14700 (Rhythm And Blues,Singer-songwriter,2010s)
Extracted 503 of 14700 (Rhythm And Blues,Alternative Rock,2010s)
Extracted 504 of 14700 (Rhythm And Blues,Indie Rock,2010s)
Extracted 505 of 14700 (Rhythm And Blues,Indie Pop,2010s)
Extracted 506 of 14700 (Rhythm And Blues,Pop Punk,2010s)
Extracted 507 of 14700 (Rhythm And Blues,Hard Rock,2010s)
Extracted 508 of 14700 (Rhythm And Blues,Contemporary Folk,2010s)
Extracted 509 of 14700 (Rhythm And Blues,Folk Rock,2010s)
Extracted 510 of 14700 (Rhythm And Blues,Folk Pop,2010s)
Extracted 511 of 14700 (Rhythm And Blues,Contemporary Country,2010s)
Extracted 512 of 14700 (Rhythm And Blues,Country Pop,2010s)
Extracted 513

Extracted 640 of 14700 (Metal,Emo-pop,2010s)
Extracted 641 of 14700 (Metal,Psychedelic Rock,2010s)
Extracted 642 of 14700 (Metal,Worship Music,2010s)
Extracted 643 of 14700 (Metal,Pop Rap,2010s)
Extracted 644 of 14700 (Metal,Psychedelic Pop,2010s)
Extracted 645 of 14700 (Metal,Blue-eyed Soul,2010s)
Extracted 646 of 14700 (Metal,Southern Rock,2010s)
Extracted 647 of 14700 (Metal,Contemporary Christian Music,2010s)
Extracted 648 of 14700 (Metal,Alternative Metal,2010s)
Extracted 649 of 14700 (Metal,Post-hardcore,2010s)
Extracted 650 of 14700 (Metal,Roots Rock,2010s)
Extracted 651 of 14700 (Metal,Heavy Metal,2010s)
Extracted 652 of 14700 (Metal,Neo-psychedelia,2010s)
Extracted 653 of 14700 (Metal,Outlaw Country,2010s)
Extracted 654 of 14700 (Metal,Glam Rock,2010s)
Extracted 655 of 14700 (Metal,Latin Pop,2010s)
Extracted 656 of 14700 (Metal,Skate Punk,2010s)
Extracted 657 of 14700 (Metal,Post-punk,2010s)
Extracted 658 of 14700 (Metal,Vocal Jazz,2010s)
Extracted 659 of 14700 (Metal,Lo-fi In

Extracted 786 of 14700 (Contemporary R&b,Psychedelic Folk,2010s)
Extracted 787 of 14700 (Contemporary R&b,Bro-country,2010s)
Extracted 788 of 14700 (Contemporary R&b,Yacht Rock,2010s)
Extracted 789 of 14700 (Contemporary R&b,Pop Reggae,2010s)
Extracted 790 of 14700 (Contemporary R&b,Alternative Dance,2010s)
Extracted 791 of 14700 (Contemporary R&b,Progressive Bluegrass,2010s)
Extracted 792 of 14700 (Contemporary R&b,Ska,2010s)
Extracted 793 of 14700 (Contemporary R&b,Anti-folk,2010s)
Extracted 794 of 14700 (Contemporary R&b,Symphonic Metal,2010s)
Extracted 795 of 14700 (Contemporary R&b,Symphonic Prog,2010s)
Extracted 796 of 14700 (Contemporary R&b,Rap Rock,2010s)
Extracted 797 of 14700 (Contemporary R&b,Comedy Rock,2010s)
Extracted 798 of 14700 (Contemporary R&b,Funk,2010s)
Extracted 799 of 14700 (Contemporary R&b,Nu Metal,2010s)
Extracted 800 of 14700 (Contemporary R&b,Celtic Punk,2010s)
Extracted 801 of 14700 (Religious Music,Pop Rock,2010s)
Extracted 802 of 14700 (Religious Music,S

Extracted 928 of 14700 (Hip Hop,Post-grunge,2010s)
Extracted 929 of 14700 (Hip Hop,Teen Pop,2010s)
Extracted 930 of 14700 (Hip Hop,Soft Rock,2010s)
Extracted 931 of 14700 (Hip Hop,Progressive Rock,2010s)
Extracted 932 of 14700 (Hip Hop,Rock And Roll,2010s)
Extracted 933 of 14700 (Hip Hop,Aor,2010s)
Extracted 934 of 14700 (Hip Hop,Christian Rock,2010s)
Extracted 935 of 14700 (Hip Hop,Britpop,2010s)
Extracted 936 of 14700 (Hip Hop,New Wave,2010s)
Extracted 937 of 14700 (Hip Hop,Piano Rock,2010s)
Extracted 938 of 14700 (Hip Hop,Heartland Rock,2010s)
Extracted 939 of 14700 (Hip Hop,Soul,2010s)
Extracted 940 of 14700 (Hip Hop,Emo-pop,2010s)
Extracted 941 of 14700 (Hip Hop,Psychedelic Rock,2010s)
Extracted 942 of 14700 (Hip Hop,Worship Music,2010s)
Extracted 943 of 14700 (Hip Hop,Pop Rap,2010s)
Extracted 944 of 14700 (Hip Hop,Psychedelic Pop,2010s)
Extracted 945 of 14700 (Hip Hop,Blue-eyed Soul,2010s)
Extracted 946 of 14700 (Hip Hop,Southern Rock,2010s)
Extracted 947 of 14700 (Hip Hop,Contem

Extracted 1087 of 14700 (Reggae,Bro-country,2010s)
Extracted 1088 of 14700 (Reggae,Yacht Rock,2010s)
Extracted 1089 of 14700 (Reggae,Pop Reggae,2010s)
Extracted 1090 of 14700 (Reggae,Alternative Dance,2010s)
Extracted 1091 of 14700 (Reggae,Progressive Bluegrass,2010s)
Extracted 1092 of 14700 (Reggae,Ska,2010s)
Extracted 1093 of 14700 (Reggae,Anti-folk,2010s)
Extracted 1094 of 14700 (Reggae,Symphonic Metal,2010s)
Extracted 1095 of 14700 (Reggae,Symphonic Prog,2010s)
Extracted 1096 of 14700 (Reggae,Rap Rock,2010s)
Extracted 1097 of 14700 (Reggae,Comedy Rock,2010s)
Extracted 1098 of 14700 (Reggae,Funk,2010s)
Extracted 1099 of 14700 (Reggae,Nu Metal,2010s)
Extracted 1100 of 14700 (Reggae,Celtic Punk,2010s)
Extracted 1101 of 14700 (Jazz,Pop Rock,2010s)
Extracted 1102 of 14700 (Jazz,Singer-songwriter,2010s)
Extracted 1103 of 14700 (Jazz,Alternative Rock,2010s)
Extracted 1104 of 14700 (Jazz,Indie Rock,2010s)
Extracted 1105 of 14700 (Jazz,Indie Pop,2010s)
Extracted 1106 of 14700 (Jazz,Pop Punk

In [78]:
len(all_songs)

26793

In [91]:
all_songs[len(all_songs)-1]

{'name': 'The Safest Place',
 'url': 'https://tabs.ultimate-guitar.com/tab/sade/the_safest_place_chords_967079',
 'genre': 'Rhythm And Blues',
 'style': 'Sophisti-pop',
 'decade': '2010s'}

In [89]:
all_combination_extractor = AllCombinationsSongExtractor(ImprovedSongExtractor)
all_songs_2 = all_combination_extractor.extract_from(1138)

cookies banner not found. Ignored
Extracted 1138 of 14700 (Jazz,Heartland Rock,2010s)
Extracted 1139 of 14700 (Jazz,Soul,2010s)
Extracted 1140 of 14700 (Jazz,Emo-pop,2010s)
Extracted 1141 of 14700 (Jazz,Psychedelic Rock,2010s)
Extracted 1142 of 14700 (Jazz,Worship Music,2010s)
Extracted 1143 of 14700 (Jazz,Pop Rap,2010s)
Extracted 1144 of 14700 (Jazz,Psychedelic Pop,2010s)
Extracted 1145 of 14700 (Jazz,Blue-eyed Soul,2010s)
Extracted 1146 of 14700 (Jazz,Southern Rock,2010s)
Extracted 1147 of 14700 (Jazz,Contemporary Christian Music,2010s)
Extracted 1148 of 14700 (Jazz,Alternative Metal,2010s)
Extracted 1149 of 14700 (Jazz,Post-hardcore,2010s)
Extracted 1150 of 14700 (Jazz,Roots Rock,2010s)
Extracted 1151 of 14700 (Jazz,Heavy Metal,2010s)
Extracted 1152 of 14700 (Jazz,Neo-psychedelia,2010s)
Extracted 1153 of 14700 (Jazz,Outlaw Country,2010s)
Extracted 1154 of 14700 (Jazz,Glam Rock,2010s)
Extracted 1155 of 14700 (Jazz,Latin Pop,2010s)
Extracted 1156 of 14700 (Jazz,Skate Punk,2010s)
Extra

Extracted 1301 of 14700 (World Music,Pop Rock,2010s)
Extracted 1302 of 14700 (World Music,Singer-songwriter,2010s)
Extracted 1303 of 14700 (World Music,Alternative Rock,2010s)
Extracted 1304 of 14700 (World Music,Indie Rock,2010s)
Extracted 1305 of 14700 (World Music,Indie Pop,2010s)
Extracted 1306 of 14700 (World Music,Pop Punk,2010s)
Extracted 1307 of 14700 (World Music,Hard Rock,2010s)
Extracted 1308 of 14700 (World Music,Contemporary Folk,2010s)
Extracted 1309 of 14700 (World Music,Folk Rock,2010s)
Extracted 1310 of 14700 (World Music,Folk Pop,2010s)
Extracted 1311 of 14700 (World Music,Contemporary Country,2010s)
Extracted 1312 of 14700 (World Music,Country Pop,2010s)
Extracted 1313 of 14700 (World Music,Electropop,2010s)
Extracted 1314 of 14700 (World Music,Indie Folk,2010s)
Extracted 1315 of 14700 (World Music,Americana,2010s)
Extracted 1316 of 14700 (World Music,Punk Rock,2010s)
Extracted 1317 of 14700 (World Music,Power Pop,2010s)
Extracted 1318 of 14700 (World Music,Country R

Extracted 1452 of 14700 (Disco,Neo-psychedelia,2010s)
Extracted 1453 of 14700 (Disco,Outlaw Country,2010s)
Extracted 1454 of 14700 (Disco,Glam Rock,2010s)
Extracted 1455 of 14700 (Disco,Latin Pop,2010s)
Extracted 1456 of 14700 (Disco,Skate Punk,2010s)
Extracted 1457 of 14700 (Disco,Post-punk,2010s)
Extracted 1458 of 14700 (Disco,Vocal Jazz,2010s)
Extracted 1459 of 14700 (Disco,Lo-fi Indie,2010s)
Extracted 1460 of 14700 (Disco,Progressive Pop,2010s)
Extracted 1461 of 14700 (Disco,Indietronica,2010s)
Extracted 1462 of 14700 (Disco,Garage Rock Revival,2010s)
Extracted 1463 of 14700 (Disco,Dream Pop,2010s)
Extracted 1464 of 14700 (Disco,Grunge,2010s)
Extracted 1465 of 14700 (Disco,Post-punk Revival,2010s)
Extracted 1466 of 14700 (Disco,Nashville Sound,2010s)
Extracted 1467 of 14700 (Disco,Jangle Pop,2010s)
Extracted 1468 of 14700 (Disco,Power Metal,2010s)
Extracted 1469 of 14700 (Disco,Celtic Rock,2010s)
Extracted 1470 of 14700 (Disco,Experimental Rock,2010s)
Extracted 1471 of 14700 (Disco

Extracted 1612 of 14700 (New Age,Country Pop,2010s)
Extracted 1613 of 14700 (New Age,Electropop,2010s)
Extracted 1614 of 14700 (New Age,Indie Folk,2010s)
Extracted 1615 of 14700 (New Age,Americana,2010s)
Extracted 1616 of 14700 (New Age,Punk Rock,2010s)
Extracted 1617 of 14700 (New Age,Power Pop,2010s)
Extracted 1618 of 14700 (New Age,Country Rock,2010s)
Extracted 1619 of 14700 (New Age,Dance-pop,2010s)
Extracted 1620 of 14700 (New Age,Blues Rock,2010s)
Extracted 1621 of 14700 (New Age,Alternative Country,2010s)
Extracted 1622 of 14700 (New Age,Adult Contemporary,2010s)
Extracted 1623 of 14700 (New Age,Synthpop,2010s)
Extracted 1624 of 14700 (New Age,Pop Soul,2010s)
Extracted 1625 of 14700 (New Age,Art Pop,2010s)
Extracted 1626 of 14700 (New Age,Art Rock,2010s)
Extracted 1627 of 14700 (New Age,Baroque Pop,2010s)
Extracted 1628 of 14700 (New Age,Post-grunge,2010s)
Extracted 1629 of 14700 (New Age,Teen Pop,2010s)
Extracted 1630 of 14700 (New Age,Soft Rock,2010s)
Extracted 1631 of 14700 (

Extracted 1766 of 14700 (Classical,Nashville Sound,2010s)
Extracted 1767 of 14700 (Classical,Jangle Pop,2010s)
Extracted 1768 of 14700 (Classical,Power Metal,2010s)
Extracted 1769 of 14700 (Classical,Celtic Rock,2010s)
Extracted 1770 of 14700 (Classical,Experimental Rock,2010s)
Extracted 1771 of 14700 (Classical,Chamber Folk,2010s)
Extracted 1772 of 14700 (Classical,Chanson,2010s)
Extracted 1773 of 14700 (Classical,Sunshine Pop,2010s)
Extracted 1774 of 14700 (Classical,Merseybeat,2010s)
Extracted 1775 of 14700 (Classical,Sophisti-pop,2010s)
Extracted 1776 of 14700 (Classical,Traditional Pop,2010s)
Extracted 1777 of 14700 (Classical,Ska Punk,2010s)
Extracted 1778 of 14700 (Classical,Neo Soul,2010s)
Extracted 1779 of 14700 (Classical,Alternative R&b,2010s)
Extracted 1780 of 14700 (Classical,Rockabilly,2010s)
Extracted 1781 of 14700 (Classical,Bluegrass,2010s)
Extracted 1782 of 14700 (Classical,Melodic Hardcore,2010s)
Extracted 1783 of 14700 (Classical,Garage Rock,2010s)
Extracted 1784 of

...

Extracted 5274 of 14700 (Reggae,Merseybeat,1990s)
Extracted 5275 of 14700 (Reggae,Sophisti-pop,1990s)
Extracted 5276 of 14700 (Reggae,Traditional Pop,1990s)
Extracted 5277 of 14700 (Reggae,Ska Punk,1990s)
Extracted 5278 of 14700 (Reggae,Neo Soul,1990s)
Extracted 5279 of 14700 (Reggae,Alternative R&b,1990s)
Extracted 5280 of 14700 (Reggae,Rockabilly,1990s)
Extracted 5281 of 14700 (Reggae,Bluegrass,1990s)
Extracted 5282 of 14700 (Reggae,Melodic Hardcore,1990s)
Extracted 5283 of 14700 (Reggae,Garage Rock,1990s)
Extracted 5284 of 14700 (Reggae,Folk Punk,1990s)
Extracted 5285 of 14700 (Reggae,Progressive Country,1990s)
Extracted 5286 of 14700 (Reggae,Psychedelic Folk,1990s)
Extracted 5287 of 14700 (Reggae,Bro-country,1990s)
Extracted 5288 of 14700 (Reggae,Yacht Rock,1990s)
Extracted 5289 of 14700 (Reggae,Pop Reggae,1990s)
Extracted 5290 of 14700 (Reggae,Alternative Dance,1990s)
Extracted 5291 of 14700 (Reggae,Progressive Bluegrass,1990s)
Extracted 5292 of 14700 (Reggae,Ska,1990s)
Extracted 

Extracted 5439 of 14700 (Blues,Soul,1990s)
Extracted 5440 of 14700 (Blues,Emo-pop,1990s)
Extracted 5441 of 14700 (Blues,Psychedelic Rock,1990s)
Extracted 5442 of 14700 (Blues,Worship Music,1990s)
Extracted 5443 of 14700 (Blues,Pop Rap,1990s)
Extracted 5444 of 14700 (Blues,Psychedelic Pop,1990s)
Extracted 5445 of 14700 (Blues,Blue-eyed Soul,1990s)
Extracted 5446 of 14700 (Blues,Southern Rock,1990s)
Extracted 5447 of 14700 (Blues,Contemporary Christian Music,1990s)
Extracted 5448 of 14700 (Blues,Alternative Metal,1990s)
Extracted 5449 of 14700 (Blues,Post-hardcore,1990s)
Extracted 5450 of 14700 (Blues,Roots Rock,1990s)
Extracted 5451 of 14700 (Blues,Heavy Metal,1990s)
Extracted 5452 of 14700 (Blues,Neo-psychedelia,1990s)
Extracted 5453 of 14700 (Blues,Outlaw Country,1990s)
Extracted 5454 of 14700 (Blues,Glam Rock,1990s)
Extracted 5455 of 14700 (Blues,Latin Pop,1990s)
Extracted 5456 of 14700 (Blues,Skate Punk,1990s)
Extracted 5457 of 14700 (Blues,Post-punk,1990s)
Extracted 5458 of 14700 (

Extracted 5591 of 14700 (World Music,Progressive Bluegrass,1990s)
Extracted 5592 of 14700 (World Music,Ska,1990s)
Extracted 5593 of 14700 (World Music,Anti-folk,1990s)
Extracted 5594 of 14700 (World Music,Symphonic Metal,1990s)
Extracted 5595 of 14700 (World Music,Symphonic Prog,1990s)
Extracted 5596 of 14700 (World Music,Rap Rock,1990s)
Extracted 5597 of 14700 (World Music,Comedy Rock,1990s)
Extracted 5598 of 14700 (World Music,Funk,1990s)
Extracted 5599 of 14700 (World Music,Nu Metal,1990s)
Extracted 5600 of 14700 (World Music,Celtic Punk,1990s)
Extracted 5601 of 14700 (Disco,Pop Rock,1990s)
Extracted 5602 of 14700 (Disco,Singer-songwriter,1990s)
Extracted 5603 of 14700 (Disco,Alternative Rock,1990s)
Extracted 5604 of 14700 (Disco,Indie Rock,1990s)
Extracted 5605 of 14700 (Disco,Indie Pop,1990s)
Extracted 5606 of 14700 (Disco,Pop Punk,1990s)
Extracted 5607 of 14700 (Disco,Hard Rock,1990s)
Extracted 5608 of 14700 (Disco,Contemporary Folk,1990s)
Extracted 5609 of 14700 (Disco,Folk Rock

Extracted 5752 of 14700 (Comedy,Neo-psychedelia,1990s)
Extracted 5753 of 14700 (Comedy,Outlaw Country,1990s)
Extracted 5754 of 14700 (Comedy,Glam Rock,1990s)
Extracted 5755 of 14700 (Comedy,Latin Pop,1990s)
Extracted 5756 of 14700 (Comedy,Skate Punk,1990s)
Extracted 5757 of 14700 (Comedy,Post-punk,1990s)
Extracted 5758 of 14700 (Comedy,Vocal Jazz,1990s)
Extracted 5759 of 14700 (Comedy,Lo-fi Indie,1990s)
Extracted 5760 of 14700 (Comedy,Progressive Pop,1990s)
Extracted 5761 of 14700 (Comedy,Indietronica,1990s)
Extracted 5762 of 14700 (Comedy,Garage Rock Revival,1990s)
Extracted 5763 of 14700 (Comedy,Dream Pop,1990s)
Extracted 5764 of 14700 (Comedy,Grunge,1990s)
Extracted 5765 of 14700 (Comedy,Post-punk Revival,1990s)
Extracted 5766 of 14700 (Comedy,Nashville Sound,1990s)
Extracted 5767 of 14700 (Comedy,Jangle Pop,1990s)
Extracted 5768 of 14700 (Comedy,Power Metal,1990s)
Extracted 5769 of 14700 (Comedy,Celtic Rock,1990s)
Extracted 5770 of 14700 (Comedy,Experimental Rock,1990s)
Extracted 5

KeyboardInterrupt: 

In [90]:
all_songs_2

[]

##  Sleep an amount of seconds to avoid service denegation

In [22]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from functools import reduce
import time
import random

class RandomWaitSongExtractor:
    BASE_URL = "https://www.ultimate-guitar.com/explore?&type[]=Chords";
    MAX_NUM_PAGES = 20
    
    def __init__(self):
        random.seed()
 
        self.driver = self.create_chrome_driver()
        self.first_time = True
    
    def get_all_songs(self, genre, style, decade):
        links = self.get_all_filter_song_links(genre['pattern'], style['pattern'], decade['pattern'])
        result = [ self.link_to_song_dict(link,genre,style,decade) for link in links]
        
        return result
    
    def link_to_song_dict(self,link, genre, style, decade):
        return {
            "name": link.contents[0],
            "url": link['href'],
            "genre": genre["name"],
            "style": style["name"],
            "decade": decade["name"]
        }
    
    def get_all_filter_song_links(self,genreFilter,styleFilter,decadeFilter):
        list_of_list = [self.get_links_single_page(genreFilter, styleFilter, decadeFilter, f'&page={page}')
                  for page in range(1,self.MAX_NUM_PAGES+1)]
        
        return reduce(lambda list1, list2: [*list1, *list2], list_of_list)
    
    def get_links_single_page(self,genreFilter,styleFilter,decadeFilter, pageFilter):
        self.wait_randomly()
        
        self.driver.get(f'{self.BASE_URL}{genreFilter}{styleFilter}{decadeFilter}')
        
        if self.driver.page_source == '<html><head></head><body></body></html>':
            raise Exception('Denegation error')

        if self.first_time:
            self.click_on_accept_cookies()
            self.first_time = False

        soup = BeautifulSoup(self.driver.page_source, 'lxml')

        return soup.findAll('a', {"class":"_2KJtL _1mes3 kWOod"})
    
    def click_on_accept_cookies(self):
        try:
            button = self.driver.find_element_by_xpath('//button[contains(text(), "thanks")]')

            button.click()
        except:
            print('cookies banner not found. Ignored')
            
    
    def create_chrome_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--headless')
        
        driver = webdriver.Chrome("./chromedriver", options=options)
        return driver
    
    def wait_randomly(self):
        n = random.random()
        time.sleep(n)

In [25]:
all_combination_extractor = AllCombinationsSongExtractor(RandomWaitSongExtractor())
all_songs = all_combination_extractor.extract_from(1)

Extracted 1 of 14700 (Rock,Pop Rock,2010s). 1000 in total
Extracted 2 of 14700 (Rock,Singer-songwriter,2010s). 2000 in total
Extracted 3 of 14700 (Rock,Alternative Rock,2010s). 3000 in total
Extracted 4 of 14700 (Rock,Indie Rock,2010s). 4000 in total
Extracted 5 of 14700 (Rock,Indie Pop,2010s). 5000 in total
Extracted 6 of 14700 (Rock,Pop Punk,2010s). 6000 in total
Extracted 7 of 14700 (Rock,Hard Rock,2010s). 7000 in total
Extracted 8 of 14700 (Rock,Contemporary Folk,2010s). 8000 in total
Extracted 9 of 14700 (Rock,Folk Rock,2010s). 9000 in total
Extracted 10 of 14700 (Rock,Folk Pop,2010s). 10000 in total
Extracted 11 of 14700 (Rock,Contemporary Country,2010s). 11000 in total
Extracted 12 of 14700 (Rock,Country Pop,2010s). 12000 in total
Extracted 13 of 14700 (Rock,Electropop,2010s). 13000 in total
Extracted 14 of 14700 (Rock,Indie Folk,2010s). 14000 in total
Extracted 15 of 14700 (Rock,Americana,2010s). 15000 in total
Extracted 16 of 14700 (Rock,Punk Rock,2010s). 16000 in total
Extrac

In [26]:
len(all_songs)

49860